<a href="https://colab.research.google.com/github/JindalDhruv05/SVNIT_NLP/blob/main/lab2/NLP_LAB2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install automathon

  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.21
    Uninstalling graphviz-0.21:
      Successfully uninstalled graphviz-0.21


In [9]:
# Install automathon first: pip install automathon

from automathon import DFA

# DFA definition
# States: q0 (start), q1 (valid), q_dead (invalid)
# Alphabet: lowercase letters a-z
# Transitions:
#   q0 --lowercase--> q1
#   q1 --lowercase--> q1
# Any other input -> q_dead

states = {'q0', 'q1'}
input_symbols = set('abcdefghijklmnopqrstuvwxyz')
transitions = {
    'q0': {ch: 'q1' for ch in input_symbols},
    'q1': {ch: 'q1' for ch in input_symbols}
}

# Start and final states
initial_state = 'q0'
final_states = {'q1'}

# Create DFA
dfa = DFA(states, input_symbols, transitions, initial_state, final_states)

# Function to check a word
def check_word(word):
    # First, reject if any char not lowercase a-z
    if not word or any(ch not in input_symbols for ch in word):
        return "Not Accepted"
    # Process in DFA
    if dfa.accept(word):
        return "Accepted"
    return "Not Accepted"

# Test cases
words = ["cat", "dog", "a", "zebra", "dog1", "1dog", "DogHouse", "Dog_house", " cats"]
for w in words:
    print(f"{w!r} -> {check_word(w)}")

# Visualization
dfa.view("dfa_diagram.png")  # Saves DFA diagram as PNG

'cat' -> Accepted
'dog' -> Accepted
'a' -> Accepted
'zebra' -> Accepted
'dog1' -> Not Accepted
'1dog' -> Not Accepted
'DogHouse' -> Not Accepted
'Dog_house' -> Not Accepted
' cats' -> Not Accepted


In [10]:
import string

# Load all nouns from the brown_nouns.txt file
with open("/content/brown_nouns.txt") as f:
    nouns = [w.strip() for w in f if w.strip()]

# Utility function: check if letter is vowel
def is_vowel(ch):
    return ch in "aeiou"

# FST simulation
def analyze_word(word):
    # Reject if contains characters outside lowercase letters
    if not word.isalpha() or not word.islower():
        return "Invalid Word"

    # Singular: no plural suffix
    if not word.endswith('s'):
        return f"{word}+N+SG"

    # Plural candidates:
    if word.endswith("ies"):
        # Rule: Y replacement: consonant + y -> ies
        if len(word) >= 4 and not is_vowel(word[-4]):
            root = word[:-3] + "y"
            return f"{root}+N+PL"
        else:
            return "Invalid Word"

    elif word.endswith("es"):
        # Rule: E insertion: after s, z, x, ch, sh
        root = word[:-2]
        if root.endswith(("s", "x", "z", "ch", "sh")):
            return f"{root}+N+PL"
        else:
            return "Invalid Word"

    else:
        # Rule: Simple S addition
        root = word[:-1]
        # Reject if it should have used 'es' or 'ies'
        if root.endswith(("s", "x", "z", "ch", "sh")):
            return "Invalid Word"
        if len(root) >= 2 and root.endswith("y") and not is_vowel(root[-2]):
            return "Invalid Word"
        return f"{root}+N+PL"

# Test with given examples
examples = ["cat", "dog", "a", "zebra", "foxes", "fox", "foxs",
            "tries", "try", "trys", "toies", "bags"]

for w in examples:
    print(f"{w} -> {analyze_word(w)}")

# Process all nouns from the corpus
results = {w: analyze_word(w) for w in nouns}

# Save output for inspection
with open("noun_analysis.txt", "w") as f:
    for w, analysis in results.items():
        f.write(f"{w} = {analysis}\n")

print("\nAnalysis complete. Results saved to noun_analysis.txt")


cat -> cat+N+SG
dog -> dog+N+SG
a -> a+N+SG
zebra -> zebra+N+SG
foxes -> fox+N+PL
fox -> fox+N+SG
foxs -> Invalid Word
tries -> try+N+PL
try -> try+N+SG
trys -> Invalid Word
toies -> Invalid Word
bags -> bag+N+PL

Analysis complete. Results saved to noun_analysis.txt


In [13]:
from graphviz import Digraph

# Create Graphviz Digraph
fst = Digraph("PluralFST", format="png")
fst.attr(rankdir="LR", size="8")

# States
states = ["q0", "q_s", "q_es", "q_ies", "q_root_SG", "q_root_PL", "q_invalid"]
accept_states = ["q_root_SG", "q_root_PL"]

for s in states:
    if s in accept_states:
        fst.attr("node", shape="doublecircle", style="filled", fillcolor="lightgreen")
    elif s == "q_invalid":
        fst.attr("node", shape="doublecircle", style="filled", fillcolor="red")
    else:
        fst.attr("node", shape="circle", style="filled", fillcolor="lightblue")
    fst.node(s)

# Transitions
# Start transitions
fst.edge("q0", "q_s", label="s")
fst.edge("q0", "q_root_SG", label="a..z except s / emit root+N+SG")

# q_s transitions
fst.edge("q_s", "q_es", label="e")
fst.edge("q_s", "q_root_PL", label="letter not requiring es or ies / emit root+N+PL")
fst.edge("q_s", "q_invalid", label="x,z,s,ch,sh or y preceded by consonant")

# q_es transitions
fst.edge("q_es", "q_ies", label="i")
fst.edge("q_es", "q_root_PL", label="x,z,s,ch,sh / emit root+N+PL")
fst.edge("q_es", "q_invalid", label="otherwise")

# q_ies transitions
fst.edge("q_ies", "q_root_PL", label="consonant / root ends with y")
fst.edge("q_ies", "q_invalid", label="vowel before y")

# Root processing loops
fst.edge("q_root_SG", "q_root_SG", label="a..z / accumulate root")
fst.edge("q_root_PL", "q_root_PL", label="a..z / accumulate root")

# Invalid loops
fst.edge("q_invalid", "q_invalid", label="a..z")

# Save and render to file in /mnt/data
output_path = "plural_fst"
fst.render(output_path, cleanup=True)
print(f"FST diagram saved to {output_path}.png")


FST diagram saved to plural_fst.png
